In [ ]:
#Change screen width
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Code to monitor local file system and initiate a transfer when a new file is created

In [ ]:
import globus_sdk
from globus_sdk.scopes import TransferScopes
from globus_sdk import TransferClient
import json
import time
import os
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

SCOPES = TransferScopes.all

# Load credentials from JSON file called config.json in the same directory as this file. The CLient_ID and ClIENT_SECRET you generate from Globus must be entered in JSON format
try:
    with open('config.json', 'r') as config_file:
        config = json.load(config_file)
        CLIENT_ID = config['CLIENT_ID']
        CLIENT_SECRET = config['CLIENT_SECRET']
except Exception as e:
    print(f"Error loading configuration: {e}")
    raise
    
def initiate_transfer(fileName):
    # Authentication
    client = globus_sdk.ConfidentialAppAuthClient(CLIENT_ID, CLIENT_SECRET)

    # Token Handling: Get Auth token and store for future use
    token_response = client.oauth2_client_credentials_tokens(requested_scopes=SCOPES)
    access_token = token_response.by_resource_server['transfer.api.globus.org']['access_token']
    authorizer = globus_sdk.AccessTokenAuthorizer(access_token=access_token)
    transfer_client = globus_sdk.TransferClient(authorizer=authorizer)

    # Set your Globus Endpoint IDs
    source_endpoint_id = "" #Enter the source collection UUID
    destination_endpoint_id = "" # Enter the the destination collection UUID

    # Build the transfer task
    task_data = globus_sdk.TransferData(
    source_endpoint=source_endpoint_id, destination_endpoint=destination_endpoint_id, label="Automation transfer", sync_level="mtime", preserve_timestamp=True, skip_source_errors=True
    )
    
    task_data.add_item(
        "/" + fileName, # source file path
        "/" + fileName,  # destination file path
    )

    # Start the transfer
    transfer_result = transfer_client.submit_transfer(task_data)
    print(f"Transfer Initiated: {transfer_result['task_id']}")
    return transfer_result['task_id']  # Return the TaskID    

# Define class that will monitor local file system
class FileTransferWatcher(FileSystemEventHandler):
    def __init__(self):
        super().__init__() 
        self.log_file = "transfer_log.txt"  # Customizable log file name

    def on_created(self, event):
        if not event.is_directory:
            filename = event.src_path  # Get the full filename 
                     
            # Wait until the file is fully written
            while True:
                try:
                    with open(filename, 'r') as f:
                        # If the file can be opened, it's likely complete
                        break  
                except OSError:
                    # File is likely being written to; wait a bit
                    time.sleep(0.5)

            print(f"File transfer complete: {filename}")
            short_filename = filename.rsplit('\\', 1)[-1]
            print(short_filename)
            task_id=initiate_transfer(short_filename)
            
            # Write to the log
            try:
                with open(self.log_file, 'a') as f:  # 'a' for append mode
                    f.write(f"{filename},{time.strftime('%Y-%m-%d %H:%M:%S')},{task_id}\n")
                    #f.write(f"{short_filename} transferred at {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
            except OSError:
                    print(f"Error writing to log file: {self.log_file}")

# Inititate FileTransferWatcher and call inititate_transfer as needed
if __name__ == "__main__":
    path_to_monitor = r''  # Update with the directory you want to monitor
    event_handler = FileTransferWatcher()
    observer = Observer()
    observer.schedule(event_handler, path_to_monitor, recursive=True)  # Recursive for subfolders
    observer.start()

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

# Check the transfer task status (high level)

In [ ]:
# Task ID to check status
TASK_ID = ''

# Create a confidential client using the Client ID and Client Secret
client = globus_sdk.ConfidentialAppAuthClient(CLIENT_ID, CLIENT_SECRET)
authorizer = globus_sdk.ClientCredentialsAuthorizer(client, SCOPES)
# Create a TransferClient object
taskStatus_client = globus_sdk.TransferClient(authorizer=authorizer)

# Get and print the task's status
task = taskStatus_client.get_task(TASK_ID)
print(f"Task Status: {task['status']}")


# Check the transfer task status (detailed)

In [ ]:
def get_transfer_results(task_id):
    client = globus_sdk.ConfidentialAppAuthClient(CLIENT_ID, CLIENT_SECRET)
    token_response = client.oauth2_client_credentials_tokens(requested_scopes=[globus_sdk.scopes.TransferScopes.all])
    access_token = token_response.by_resource_server['transfer.api.globus.org']['access_token']
    authorizer = globus_sdk.AccessTokenAuthorizer(access_token=access_token)
    transfer_client = globus_sdk.TransferClient(authorizer=authorizer)

    # Get detailed task information975745c0-49d5-11ef-b6c5-f55c894bd1c6
    task = transfer_client.get_task(task_id).data
    print("Task Information:")
    print(json.dumps(task, indent=2))

    # Get list of events for the task
    events_response = transfer_client.task_event_list(task_id)
    print("Events Response Data:")
    print(json.dumps(events_response.data, indent=2))

    events = events_response['DATA']
    print("\nTask Events:")
    for event in events:
        print(json.dumps(event, indent=2))

    # Get list of successful files
    successful_files_response = transfer_client.task_successful_transfers(task_id)
    print("Successful Files Response Data:")
    print(json.dumps(successful_files_response.data, indent=2))

    successful_files = successful_files_response['DATA']
    print("\nSuccessful Files:")
    for file in successful_files:
        print(json.dumps(file, indent=2))

    # Get list of skipped errors
    skipped_errors_response = transfer_client.task_skipped_errors(task_id)
    print("Skipped Errors Response Data:")
    print(json.dumps(skipped_errors_response.data, indent=2))

    skipped_errors = skipped_errors_response['DATA']
    print("\nSkipped Errors:")
    for error in skipped_errors:
        print(json.dumps(error, indent=2))


get_transfer_results(TASK_ID)